<a href="https://colab.research.google.com/github/mphirke/fire-emblem-fake-portaits-GBA/blob/master/Fire_Emblem_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2: training a model from scratch

References:
- Steam StyleGAN2 by woctezuma: https://github.com/woctezuma/steam-stylegan2
- the original StyleGAN2 repository: https://github.com/NVlabs/stylegan2
- woctezuma's fork of StyleGAN2: https://github.com/woctezuma/stylegan2 for easy saving results in google drive.


### Request more RAM (once per session)

To have access to more memory, crash the Google Colab session  once.

References:
-   https://github.com/googlecolab/colabtools/issues/253#issuecomment-551056637
-   https://colab.research.google.com/drive/1dBN-wwYUngLYVt985wGs_OKPlK_ANB9D

It might not be required but if your runtime crashes, 
uncomment and run the following cell if you do not have Colab high RAM (26 GB).

In [1]:
# d = [ '0' ]
# while(True):
#   d += d

### Check the GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-9b65c71e-72e9-9702-4f91-2bb9dcf5e778)


### Switch to Tensorflow 1.x

The default TensorFlow version in Colab switched from  1.x to 2.x on the 27th of March, 2020.

Reference: https://colab.research.google.com/notebooks/tensorflow_version.ipynb

We switch to version 1.x to avoid the following error:
> `ModuleNotFoundError: No module named 'tensorflow.contrib' colab`

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
!wget https://www.dropbox.com/s/gt302rbcri7ktgf/FE_Portraits_one_folder_cropped.zip?dl=1
!unzip -o FE_Portraits_one_folder_cropped.zip?dl=1

--2020-07-26 17:40:15--  https://www.dropbox.com/s/gt302rbcri7ktgf/FE_Portraits_one_folder_cropped.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gt302rbcri7ktgf/FE_Portraits_one_folder_cropped.zip [following]
--2020-07-26 17:40:15--  https://www.dropbox.com/s/dl/gt302rbcri7ktgf/FE_Portraits_one_folder_cropped.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9969c15636a9df2ff00c43a609.dl.dropboxusercontent.com/cd/0/get/A8TF0At8XSKah_mhWVwkZUpgY-JyH_7TaL4hof-AfYzDgxOQCqhrnYv-HrH7uSNhb3I7dSOOQZLRcnlstVcGpliiyXEB3Em-pYm6sDVcNiZe1LvuubDZHgZOzN2pSL_Qmy0/file?dl=1# [following]
--2020-07-26 17:40:16--  https://uc9969c15636a9df2ff00c43a609.dl.dropboxusercontent.com/cd/0/get/A8TF0At8XSKah_mhWVwkZUpgY-JyH_7TaL4hof-A

## Installing StyleGAN2

### woctezuma's fork of the official StyleGAN2 implementation

In [6]:
%cd /content/

/content


In [7]:
!rm -rf stylegan2/
!git clone https://github.com/woctezuma/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 120, done.
remote: Total 120 (delta 0), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (120/120), 586.40 KiB | 815.00 KiB/s, done.
Resolving deltas: 100% (51/51), done.


In [8]:
pushd

['/content']

In [9]:
%cd stylegan2/

/content/stylegan2


Switch to one of my branches, designed for Google Colab:
-   `google-colab` to save every 2 ticks,
-   `google-colab-save-every-tick` to save every tick.

NB: It is expected that 1 tick ~ 15 min. However, if you are unlucky with the Colab machine which is allotted, 1 tick ~ 1 hour.

In [10]:
!git checkout google-colab-save-every-tick

Branch 'google-colab-save-every-tick' set up to track remote branch 'google-colab-save-every-tick' from 'origin'.
Switched to a new branch 'google-colab-save-every-tick'


In [11]:
!nvcc test_nvcc.cu -o test_nvcc -run

CPU says hello.
GPU says hello.


In [12]:
popd

/content
popd -> /content


## Mounting Google Drive

In [13]:
!pip install Google-Colab-Transfer

In [14]:
import colab_transfer

colab_path = colab_transfer.get_path_to_home_of_local_machine()
drive_path = colab_transfer.get_path_to_home_of_google_drive()

print('Disk of the virtual matchine: {}'.format(colab_path))
print('Google Drive: {}'.format(drive_path))

Disk of the virtual matchine: /content/
Google Drive: /content/drive/My Drive/


In [16]:
colab_transfer.mount_google_drive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Data parameters

In [17]:
folder_name = '/content/FE_Portraits_one_folder'

## Preparing datasets (once)

### Preparing data for StyleGAN2

In [24]:
!rm /content/FE_Portraits_one_folder_cropped
!python stylegan2/dataset_tool.py create_from_images '/content/datasets/steam' '/content/FE_Portraits_one_folder_cropped'

rm: cannot remove '/content/FE_Portraits_one_folder_cropped': Is a directory
Loading images from "/content/FE_Portraits_one_folder_cropped"
Creating dataset "/content/datasets/steam"
Added 1153 images.


### Archive prepared datasets to Google Drive

In [21]:
# colab_transfer.copy_folder(folder_name,
#                            source=colab_path,
#                            destination=drive_path)

## Importing prepared datasets from Google Drive

In [22]:
# colab_transfer.copy_folder(folder_name,
#                            source=drive_path,
#                            destination=colab_path)

Copying /content/drive/My Drive//content/FE_Portraits_one_folder to /content//content/FE_Portraits_one_folder
Folder /content/drive/My Drive//content/FE_Portraits_one_folder could not be found. Copy aborted.


## Training networks

There is no need to edit `training/training_loop.py`, thanks to automatic resuming from the latest snapshot, implemented in my fork.

Otherwise, one would have to manually edit the file from within the Google Colab session!

### Train with the official StyleGAN2 implementation

- `--mirror-augment=true`: data augmentation with horitontal mirroring,
- `--total-kimg=5000`: 5000k images trained

Model snapshots are directly saved to Google Drive (`--result-dir='/content/drive/My Drive/results'`).

In [ ]:
!python stylegan2/run_training.py --config=config-e --metrics=none \
   --data-dir='/content/datasets' --dataset=steam \
   --mirror-augment=true \
   --total-kimg=5000 \
   --result-dir='/content/drive/My Drive/results' \


Local submit - run_dir: /content/drive/My Drive/results/00002-stylegan2-steam-1gpu-config-e
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x79ee000 @  0x7f01ecd3a001 0x7f01e97c6765 0x7f01e982abb0 0x7f01e982ca4f 0x7f01e98c3048 0x50a635 0x50cd96 0x507d64 0x509042 0x594931 0x549e5f 0x5513d1 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x588e5c 0x59fc4e
tcmalloc: large alloc 4294967296 bytes == 0x7f00242c6000 @  0x7f01ecd381e7 0x7f01e97c65e1 0x7f01e982ac78 0x7f01e982af37 0x7f01e98c2f28 0x50a635 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64
tcmalloc: large alloc 4294967296 bytes == 0x7ef